载入函数库

In [3]:
import numpy as np
import pandas as pd
import sys 
import matplotlib.pyplot as plt
import sys
sys.path.append('E:\\python\\python FMR_analyse\\FMR__Songqi')
import sqa
reload(sqa) 
import statsmodels.api as sm
%matplotlib qt


拟合Lorentz

In [35]:
#line shape fitting
table_lineshape_fit=sqa.origin_creat_table('line100k_9T')
longname_unit_comment1=[['sysmetrical height','',''],['resonance field','',''],['half linewidth','',''],['asysmetrical height','',''],
                        ['linear slope','',''],['constant','','']]

#folder='E:\\project\\S-F-S\\Nb-Py-Nb\\FMR\\16nm\\16nm-4'
folder='E:\\project\\Nontrival SC\\YBCO-YIG\\FMR\\YIG-YBCO-50K\\YBCO-Py\\Py +15nm'
filename='1st_T=100K_5dBm_S21_9T Cool.txt'
data=sqa.fmr_load_data(folder+'\\'+filename)
n=0
result=[]
for i in data.columns:
    frequency=float(i[1:])
    figure_path=folder+'\\'+filename.replace('.txt','freq'+str(float(i[1:])/1e9)+'GHz with_substract.png')
    a=sqa.fmr_select_frequency(data,frequency)
    a_new=a.values
    #a_new=sqa.substrate_background_from_one_curve(a.index.values,a.values)
    fitting_para=sqa.fit_one_curve(a.index.values,a_new,figure_path=figure_path)
    #print parameters
    print [n,0],frequency
    n=n+1
    if sum(abs(fitting_para))!=0:
        sqa.origin_get_array(table_lineshape_fit,np.array(frequency,ndmin=2),position=[n,0],longname_unit_comment=[['frequency(Hz)','','']])
        sqa.origin_get_array(table_lineshape_fit,np.array(fitting_para,ndmin=2),position=[n,1],longname_unit_comment=longname_unit_comment1)
        result.append([frequency]+fitting_para.tolist())


		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		
[0, 0] 1000000000.0

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		
[1, 0] 1250000000.0

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		
[2, 0] 1500000000.0

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		

		no point => ok 
		1 point => center
		3 point => limit the left and right width
		>4 point => repeat
        5 point => give up
		
[3

fitting_para.tolist()?

拟合Damping

In [51]:
a_new=sqa.substrate_background_from_one_curve(a.index.values,a.values)

error: (m>k) failed for hidden m: fpcurf0:m=3

In [71]:
tolist()

SyntaxError: invalid syntax (<ipython-input-71-ec5bc7882864>, line 1)

In [36]:
#damping fit`
gamma=1.76*10**10
result=np.array(result)
plt.plot(result[:,0],result[:,3],'.')
plt.draw()
plt.pause(0.1)

while True:
    ginput=plt.ginput(0)
    if len(ginput)==0:
        break
        print 'ok'
    if len(ginput)==2:
        plt.clf()
        temp=result[:,[0,3]]
        temp=temp[temp[:,0]>=ginput[0][0]]
        temp=temp[temp[:,0]<=ginput[1][0]]
        mod = sm.OLS(temp[:,1], sm.add_constant(temp[:,0]))
        res = mod.fit()
        linear_fit_result=np.array([res.params[1],res.params[0]])
        
        
        confidence_interval=res.conf_int(0.05)[1]
        plt.plot(result[:,0],result[:,3],'.')
        fitx=np.linspace(temp[0,0],temp[-1,0],100)
        fity=np.polyval(linear_fit_result,fitx)
        plt.plot(fitx,fity,'.')
        plt.draw()
        plt.pause(0.1)
        damping=linear_fit_result[0]*gamma/(2*3.1415926)/1000
        damping_interval=confidence_interval*gamma/(2*3.1415926)/1000
        print 'Gilbert Damping : ', damping
        print 'Gilbert Damping Confidence Interval(0.05): ', damping_interval[1]-damping_interval[0] 

table_damping_fit=sqa.origin_creat_table('Damping_300K')
sqa.origin_get_array(\
                    table_damping_fit,temp,position=[0,0],\
                    longname_unit_comment=[['frequency','(Hz)',''],['Half-Linewidth','Oe','']])
sqa.origin_get_array(\
                     table_damping_fit,np.array([fitx,fity]).T,position=[0,2],\
                     longname_unit_comment=[['fit_freq','(Hz)',''],['fit_LW','Oe','']])


Gilbert Damping :  0.0107917074149
Gilbert Damping Confidence Interval(0.05):  0.000784425844573


拟合Kittel

In [7]:
?res.conf_int(0.05)

In [37]:
#Kittel fit
frequency_array=result[:,0]
resonance_field_array=result[:,2]

frequency_trans_array=(2*3.1415*frequency_array/(1.76*10**7))**2
fit_para=np.polyfit(resonance_field_array,frequency_trans_array,2)
pi4Meff=fit_para[1]
print '4*Pi*Meff:',pi4Meff,'Gs'

#plot

plt.figure()
plt.plot(resonance_field_array,frequency_array,'o',linewidth=20)
fit_meff_x=np.linspace(0,resonance_field_array[-1],100)
fit_meff_y=(fit_meff_x**2*fit_para[0]+fit_meff_x*fit_para[1]+fit_para[2])**0.5*1.76*1e7/(2*3.1415)
plt.plot(fit_meff_x,fit_meff_y,'.')

#write into origin
table_kittel_fit=sqa.origin_creat_table('Kittel_300K')
sqa.origin_get_array(\
                    table_kittel_fit,np.array([resonance_field_array,frequency_array]).T\
                    ,position=[0,0],longname_unit_comment=[['H_res','(Oe)',''],['f_res','(Hz)','']])
sqa.origin_get_array(\
                     table_kittel_fit,np.array([fit_meff_x,fit_meff_y]).T,position=[0,2],\
                     longname_unit_comment=[['H_fit','(Oe)',''],['fit_f','Hz','']])

4*Pi*Meff: 8687.82627188 Gs






#Non-linear linewidth vs Frequency

In [104]:
from scipy.optimize import least_squares
gamma=1.76*10**10
def fit_guassion_robust(fitx,fity,x0=False):
    '''
    p[0] damping
    p[1] H_0
    p[2] A
    p[3] tau
    '''
    def fun_fit(p,x,y):
        return fun_cal_y(p,x)-y 
    def fun_cal_y(p,x):
        return 2*3.1415926*p[0]*x*1000/gamma+p[1]+p[2]*(2*3.1415926*x*p[3])/(1+(2*3.1415926*x*p[3])**2)
     
    if not x0: 
        x0=[0.05,0,100,1e-13] 
    else: 
        x0=x0
        
    res_robust=least_squares(fun_fit,x0,args=(fitx,fity),\
                             bounds=([0,-np.inf,-np.inf,1e-19],[np.inf,np.inf,np.inf,3.9e-11]))
    fitted_y=fun_cal_y(res_robust.x,fitx)
    return res_robust.x,fitted_y

#damping fit
gamma=1.76*10**10
#lw_freq=np.array(result)[:,[0,3]]
lw_freq=np.loadtxt('C:\\Users\\user\\Desktop\\2-15nm\\HALF\\4K.txt')
plt.plot(lw_freq[:,0],lw_freq[:,1],'.')
plt.draw()
plt.pause(0.1)

while True:
    ginput=plt.ginput(0)
    if len(ginput)==0:
        break
        #print 'ok'
    if len(ginput)==2:
        plt.clf()
        temp=lw_freq
        temp=temp[temp[:,0]>=ginput[0][0]]
        temp=temp[temp[:,0]<=ginput[1][0]]
        
        fit_res,fity=fit_guassion_robust(temp[:,0], temp[:,1])
        print('fit_res',fit_res)
        print('temp',temp)
        print('fity',fity)
        
        
        plt.plot(lw_freq[:,0],lw_freq[:,1],'.')
        plt.plot(temp[:,0],fity,'.')
        plt.draw()
        plt.pause(0.1)
        
        

('fit_res', array([  3.02845636e-02,   8.65483095e+01,   1.50093886e+02,
         1.85730489e-11]))


In [82]:
def fun_cal_y(p,x):
    return 2*3.1415926*p[0]*x*1000/gamma+p[1]+p[2]*(2*3.1415926*x*p[3])/(1+(2*3.1415926*x*p[3])**2)
x=np.linspace(0,1e11,1000)
p=fit_res
plt.plot(x,fun_cal_y(p,x))

In [83]:
#output the two-magnon fitting results

pd.DataFrame(fun_cal_y(p,x),index=x).to_csv('C:\\Users\\user\\Desktop\\2-15nm\\HALF\\6-2.txt')

In [37]:
p_5=np.array([  0.02765,-27.74951,502.24709,1.44349E-11])
p_10=np.array([ 0.01827,3.14391,428.89911,1.63975E-11])
p_20=np.array([ 0.00826,45.61887,453.41117,1.36642E-11])
p_30=np.array([ 0.00579,-10.81454,596.66453,1.44787E-11])

x=np.linspace(0,1e11,100)

plt.plot(x,fun_cal_y(p_5,x))
plt.plot(x,fun_cal_y(p_10,x))
plt.plot(x,fun_cal_y(p_20,x))
plt.plot(x,fun_cal_y(p_30,x))


In [57]:
 res_robust=least_squares(fun_fit,x0,args=(fitx,fity),diff_step=np.array([1e-5,1e-5,1e-5,1e-16]),\
                             x_scale=np.array([1,1,1,1e-10]),f_scale=100,loss='cauchy',\
                             bounds=([0,-np.inf,-np.inf,0],[np.inf,np.inf,np.inf,3e-10]))

TypeError: data type not understood

In [ ]:
# NEWONE TO FITTING --OPTIIMAL BY QI

In [230]:

from scipy.optimize import least_squares
gamma=1.76*10**10
def fit_guassion_robust(fitx,fity,x0=False):
    '''
    p[0] damping
    p[1] H_0
    p[2] A
    p[3] tau
    '''
    def fun_fit(p,x,y):
        return fun_cal_y(p,x)-y 
    def fun_cal_y(p,x):
        return 2*3.1415926*p[0]*x*1000/gamma+p[1]+p[2]*(2*3.1415926*x*p[3])/(1+(2*3.1415926*x*p[3])**2)
     
    if not x0: 
        x0=[0.05,0,100,1e-13] 
    else: 
        x0=x0
        
    res_robust=least_squares(fun_fit,x0,args=(fitx,fity),diff_step=np.array([1e-5,1e-5,1e-5,1e-16]),\
                             x_scale=np.array([1,1,1,1e-10]),f_scale=100,loss='cauchy',\
                             bounds=([0,-np.inf,-np.inf,0],[np.inf,np.inf,np.inf,1e-10]))
    fitted_y=fun_cal_y(res_robust.x,fitx)
    return res_robust.x,fitted_y

#damping fit
gamma=1.76*10**10
#lw_freq=np.array(result)[:,[0,3]]
lw_freq=np.loadtxt('C:\\Users\\user\\Desktop\\2_20\\FINAL\\11K.txt')
plt.plot(lw_freq[:,0],lw_freq[:,1],'.')
plt.draw()
plt.pause(0.1)

while True:
    ginput=plt.ginput(0)
    if len(ginput)==0:
        break
        #print 'ok'
    if len(ginput)==2:
        plt.clf()
        temp=lw_freq
        temp=temp[temp[:,0]>=ginput[0][0]]
        temp=temp[temp[:,0]<=ginput[1][0]]
        #print('temp',temp)
        fit_res,fity=fit_guassion_robust(temp[:,0], temp[:,1])
        print('fit_res',fit_res)
        #print('fity',fity)
        
        plt.plot(lw_freq[:,0],lw_freq[:,1],'.')
        plt.plot(temp[:,0],fity,'.')
        plt.draw()
        plt.pause(0.1)
        

('fit_res', array([  2.26528219e-02,  -1.51165515e+00,   4.50222995e+02,
         1.76088890e-11]))


In [193]:
def fun_cal_y(p,x):
    return 2*3.1415926*p[0]*x*1000/gamma+p[1]+p[2]*(2*3.1415926*x*p[3])/(1+(2*3.1415926*x*p[3])**2)
x=np.linspace(0,1e11,1000)
p=fit_res
plt.plot(x,fun_cal_y(p,x))

In [194]:
#output the two-magnon fitting results

pd.DataFrame(fun_cal_y(p,x),index=x).to_csv('C:\\Users\\user\\Desktop\\2_20\\RESULTS\\11K.txt')